In [1]:
#import required datasets
import requests
import pandas as pd
from bs4 import BeautifulSoup
import folium
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
from geopy.geocoders import Nominatim

# 1. Scraping the Data from the Wikipedia Page

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html_data = requests.get(url).text
soup = BeautifulSoup(html_data, 'html5lib')

### 1.1 - Cleaning and wrangling the data

In [3]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

### 1.2 - Checking for not assigned data

In [4]:
print(f"There are {df.loc[df['Borough'] == 'Not assigned'].shape[0]} Not assigned Boroughs")

There are 0 Not assigned Boroughs


In [5]:
print(f"There are {df.loc[df['Neighborhood'] == 'Not assigned'].shape[0]} Not assigned Neighborhoods")

There are 0 Not assigned Neighborhoods


In [6]:
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


### 1.3 - Testing if the example matches the data

In [7]:
example_postalcode = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]
df.loc[df['PostalCode'].isin(example_postalcode)]

,PostalCode,Borough,Neighborhood
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
6,M1B,Scarborough,"Malvern, Rouge"
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
23,M4G,East York,Leaside
24,M5G,Downtown Toronto,Central Bay Street
27,M2H,North York,Hillcrest Village
32,M1J,Scarborough,Scarborough Village
50,M9L,North York,Humber Summit
54,M4M,East Toronto,Studio District
71,M1R,Scarborough,"Wexford, Maryvale"


### 1.4 - Printing the shape of the data

In [8]:
df.shape

(103, 3)

# 2. Getting the geographical coordinates from the data

In [9]:
geospatial_url = "https://cocl.us/Geospatial_data"
geospatial_data = pd.read_csv(geospatial_url)

In [10]:
geospatial_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Merge two dataframes

In [11]:
geospatial_data.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
df = df.join(geospatial_data.set_index('PostalCode'), on='PostalCode')

In [12]:
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


### 2.1 - Testing if the example matches the data

In [13]:
example_postalcode = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]
df.loc[df['PostalCode'].isin(example_postalcode)]

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
23,M4G,East York,Leaside,43.709060,-79.363452
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
27,M2H,North York,Hillcrest Village,43.803762,-79.363452
32,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
50,M9L,North York,Humber Summit,43.756303,-79.565963
54,M4M,East Toronto,Studio District,43.659526,-79.340923
71,M1R,Scarborough,"Wexford, Maryvale",43.750072,-79.295849


# 3. Explore and cluster the neighborhoods in Toronto

### 3.1 - Creating map of all the city with neighborhoods superimposed on top

Getting the location of toronto with geopy

In [14]:
address = 'Toronto'

geolocator = Nominatim(user_agent="toronto-explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [16]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### 3.2 - Selecting only the boroughs that contain the word Toronto

In [17]:
df_toronto_borough = df[df['Borough'].str.contains("Toronto")].reset_index(drop=True)
df_toronto_borough.shape

(39, 5)

### 3.3 - Creating map of toronto borough with neighborhoods superimposed on top

In [18]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto_borough['Latitude'], df_toronto_borough['Longitude'], df_toronto_borough['Borough'], df_toronto_borough['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### 3.4. Use the Foursquare API to explore the neighborhoods

In [19]:
CLIENT_ID = 'NP0EEJ1YUX1SZ41YLIRQOJUEHBDTKBIBM110Z34ZMXWK12SG' # your Foursquare ID
CLIENT_SECRET = 'TDOXG2NDZ5Q4POGXYIY032KL0LYLKTEPPAETWVFTUVGWSFIA' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: NP0EEJ1YUX1SZ41YLIRQOJUEHBDTKBIBM110Z34ZMXWK12SG
CLIENT_SECRET:TDOXG2NDZ5Q4POGXYIY032KL0LYLKTEPPAETWVFTUVGWSFIA


In [20]:
df_toronto_borough.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


### 3.5 - Let's get all the venues by borough and neighborhood

In [21]:
def getNearbyVenues(boroughs, neighborhoods, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for borough, neighborhood, lat, lng in zip(boroughs, neighborhoods, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            borough,
            neighborhood, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = [
                  'Borough',
                  'Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [22]:
borough_neighborhood_venues = getNearbyVenues(boroughs=df_toronto_borough['Borough'],
                                             neighborhoods=df_toronto_borough['Neighborhood'],
                                             latitudes=df_toronto_borough['Latitude'],
                                             longitudes=df_toronto_borough['Longitude'])

In [23]:
borough_neighborhood_venues.head()

,Borough,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
4,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


### 3.6 - Checking the size of the dataframe

In [24]:
borough_neighborhood_venues.shape

(1600, 8)

In [25]:
borough_neighborhood_venues.groupby(['Borough', 'Neighborhood']).count()

Neighborhood Latitude  \
Borough                Neighborhood                                                                
Central Toronto        Davisville                                                             36   
                       Davisville North                                                        9   
                       Forest Hill North & West                                                4   
                       Lawrence Park                                                           3   
                       Moore Park, Summerhill East                                             2   
                       North Toronto West                                                     19   
                       Roselawn                                                                2   
                       Summerhill West, Rathnelly, South Hill, Forest ...                     13   
                       The Annex, North Midtown, Yorkville                                    19   
Downtown Toronto       Berczy Park                                                            59   
                       CN Tower, King and Spadina, Railway Lands, Harb...                     18   
                       Central Bay Street                                                     68   
                       Christie                                                               16   
                       Church and Wellesley                                                   79   
                       Commerce Court, Victoria Hotel                                        100   
                       First Canadian Place, Underground city                                100   
                       Garden District, Ryerson                                              100   
                       Harbourfront East, Union Station, Toronto Islands                     100   
                       Kensington Market, Chinatown, Grange Park                              66   
                       Regent Park, Harbourfront                                              45   
                       Richmond, Adelaide, King                                              100   
                       Rosedale                                                                4   
                       St. James Town                                                         87   
                       St. James Town, Cabbagetown                                            45   
                       Toronto Dominion Centre, Design Exchange                              100   
                       University of Toronto, Harbord                                         33   
Downtown Toronto Stn A Enclave of M5E                                                         98   
East Toronto           India Bazaar, The Beaches West                                         19   
                       Studio District                                                        36   
                       The Beaches                                                             4   
                       The Danforth West, Riverdale                                           42   
East Toronto Business  Enclave of M4L                                                         15   
East York/East Toronto The Danforth  East                                                      4   
West Toronto           Brockton, Parkdale Village, Exhibition Place                           25   
                       Dufferin, Dovercourt Village                                           15   
                       High Park, The Junction South                                          25   
                       Little Portugal, Trinity                                               42   
                       Parkdale, Roncesvalles                                                 14   
                       Runnymede, Swansea                                  

#### Let's find out how many unique categories can be curated from all the returned venues

In [26]:
print(f'There are {borough_neighborhood_venues["Venue Category"].nunique()} uniques categories.')

There are 234 uniques categories.


### 3.7 - Analyze each Neighborhood and Venue

In [27]:
# one hot encoding
toronto_onehot = pd.get_dummies(borough_neighborhood_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Borough'] = borough_neighborhood_venues['Borough'] 
toronto_onehot['Neighborhoods'] = borough_neighborhood_venues['Neighborhood'] 
toronto_onehot['Venue'] = borough_neighborhood_venues['Venue'] 

# move neighborhood column to the first column
fixed_columns = list(toronto_onehot.columns[-3:]) + list(toronto_onehot.columns[:-3])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Borough,Neighborhoods,Venue,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,Downtown Toronto,"Regent Park, Harbourfront",Roselle Desserts,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Downtown Toronto,"Regent Park, Harbourfront",Tandem Coffee,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Downtown Toronto,"Regent Park, Harbourfront",Cooper Koo Family YMCA,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Downtown Toronto,"Regent Park, Harbourfront",Impact Kitchen,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Downtown Toronto,"Regent Park, Harbourfront",Body Blitz Spa East,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
toronto_onehot.shape

(1600, 237)

#### Next, let's group rows by borough and neighborhood and by taking the mean of the frequency of occurrence of each category

In [29]:
toronto_grouped = toronto_onehot.groupby(['Borough', 'Neighborhoods']).mean().reset_index()
toronto_grouped

,Borough,Neighborhoods,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,Central Toronto,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.027778,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
1,Central Toronto,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
2,Central Toronto,Forest Hill North & West,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.25000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
3,Central Toronto,Lawrence Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Toronto,"Moore Park, Summerhill East",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.50000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
5,Central Toronto,North Toronto West,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.052632
6,Central Toronto,Roselawn,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
7,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.076923,0.000000,...,0.000000,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.076923,0.000000,0.000000
8,Central Toronto,"The Annex, North Midtown, Yorkville",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
9,Downtown Toronto,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.00000,0.00,0.016949,0.000000,0.000000,0.000000,0.000000


In [30]:
toronto_grouped.shape

(39, 236)

### Let's print each neighborhood along with the top 5 most common venues

In [31]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhoods']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhoods'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[2:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Davisville----
                venue  freq
0        Dessert Shop  0.08
1         Pizza Place  0.08
2      Sandwich Place  0.08
3     Thai Restaurant  0.06
4  Italian Restaurant  0.06


----Davisville North----
                  venue  freq
0        Breakfast Spot  0.11
1  Gym / Fitness Center  0.11
2      Department Store  0.11
3          Dance Studio  0.11
4                 Hotel  0.11


----Forest Hill North & West----
              venue  freq
0             Trail  0.25
1          Bus Line  0.25
2  Sushi Restaurant  0.25
3     Jewelry Store  0.25
4           Airport  0.00


----Lawrence Park----
         venue  freq
0         Park  0.33
1     Bus Line  0.33
2  Swim School  0.33
3      Airport  0.00
4       Museum  0.00


----Moore Park, Summerhill East----
                 venue  freq
0                Trail   0.5
1          Summer Camp   0.5
2              Airport   0.0
3               Museum   0.0
4  Martial Arts School   0.0


----North Toronto West----
                 venue  

#### Let's put that into a pandas dataframe

First, let's write a function to sort the venues in descending order.

In [32]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[2:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [33]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Borough', 'Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Borough'] = toronto_grouped['Borough']
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhoods']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 2:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Davisville,Pizza Place,Dessert Shop,Sandwich Place,Gym,Café,Coffee Shop,Italian Restaurant,Thai Restaurant,Sushi Restaurant,Fried Chicken Joint
1,Central Toronto,Davisville North,Pizza Place,Hotel,Gym / Fitness Center,Park,Breakfast Spot,Sandwich Place,Department Store,Food & Drink Shop,Dance Studio,Doner Restaurant
2,Central Toronto,Forest Hill North & West,Trail,Jewelry Store,Bus Line,Sushi Restaurant,Yoga Studio,Distribution Center,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room
3,Central Toronto,Lawrence Park,Park,Bus Line,Swim School,Yoga Studio,Distribution Center,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store
4,Central Toronto,"Moore Park, Summerhill East",Trail,Summer Camp,Yoga Studio,Discount Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store


In [34]:
neighborhoods_venues_sorted.shape

(39, 12)

### 3.8. Cluster Neighborhoods 

#### Run k-means to cluster the neighborhood into 5 clusters.

In [35]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop(['Borough', 'Neighborhoods'], 1)

# run k-means clustering
kmeans = KMeans(init='k-means++', n_clusters=kclusters, random_state=1, max_iter=1000).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 4, 1, 3, 0, 2, 0, 0, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [36]:
# add clustering labels
neighborhoods_venues_sorted['Cluster Labels'] = kmeans.labels_

toronto_merged = df_toronto_borough.copy()

# merge toronto_grouped with df_toronto_borough to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.drop(['Borough'], axis=1).set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,Coffee Shop,Bakery,Pub,Park,Café,Breakfast Spot,Theater,Chocolate Shop,Distribution Center,Mexican Restaurant,0
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,Coffee Shop,Clothing Store,Japanese Restaurant,Bubble Tea Shop,Café,Cosmetics Shop,Ramen Restaurant,Pizza Place,Fast Food Restaurant,Hotel,0
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,Coffee Shop,Café,Restaurant,Clothing Store,Cocktail Bar,Cosmetics Shop,Bakery,Seafood Restaurant,Park,Creperie,0
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,Health Food Store,Neighborhood,Trail,Pub,Yoga Studio,Dumpling Restaurant,Dog Run,Doner Restaurant,Donut Shop,Electronics Store,0
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,Coffee Shop,Cocktail Bar,Bakery,Farmers Market,Seafood Restaurant,Restaurant,Beer Bar,Cheese Shop,Pharmacy,Pub,0


#### Finally, let's visualize the resulting clusters

In [37]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### 3.9 - Examining the Clusters

#### Cluster - 1

In [38]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
0,Downtown Toronto,Coffee Shop,Bakery,Pub,Park,Café,Breakfast Spot,Theater,Chocolate Shop,Distribution Center,Mexican Restaurant,0
1,Downtown Toronto,Coffee Shop,Clothing Store,Japanese Restaurant,Bubble Tea Shop,Café,Cosmetics Shop,Ramen Restaurant,Pizza Place,Fast Food Restaurant,Hotel,0
2,Downtown Toronto,Coffee Shop,Café,Restaurant,Clothing Store,Cocktail Bar,Cosmetics Shop,Bakery,Seafood Restaurant,Park,Creperie,0
3,East Toronto,Health Food Store,Neighborhood,Trail,Pub,Yoga Studio,Dumpling Restaurant,Dog Run,Doner Restaurant,Donut Shop,Electronics Store,0
4,Downtown Toronto,Coffee Shop,Cocktail Bar,Bakery,Farmers Market,Seafood Restaurant,Restaurant,Beer Bar,Cheese Shop,Pharmacy,Pub,0
5,Downtown Toronto,Coffee Shop,Café,Sandwich Place,Italian Restaurant,Restaurant,Bubble Tea Shop,Japanese Restaurant,Burger Joint,Spa,Salad Place,0
6,Downtown Toronto,Grocery Store,Café,Park,Nightclub,Candy Store,Baby Store,Italian Restaurant,Athletics & Sports,Restaurant,Coffee Shop,0
7,Downtown Toronto,Coffee Shop,Café,Clothing Store,Restaurant,Hotel,Bar,Thai Restaurant,Gym,Pizza Place,Sushi Restaurant,0
8,West Toronto,Pharmacy,Bakery,Middle Eastern Restaurant,Music Venue,Park,Grocery Store,Pizza Place,Café,Brewery,Bar,0
9,East York/East Toronto,Intersection,Convenience Store,Park,Coffee Shop,Electronics Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,0


#### Cluster - 2

In [39]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
18,Central Toronto,Park,Bus Line,Swim School,Yoga Studio,Distribution Center,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,1
33,Downtown Toronto,Park,Playground,Trail,Yoga Studio,Diner,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,1


#### Cluster - 3

In [40]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
19,Central Toronto,Garden,Home Service,Yoga Studio,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,2


#### Cluster - 4

In [41]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
29,Central Toronto,Trail,Summer Camp,Yoga Studio,Discount Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,3


#### Cluster - 5

In [42]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
21,Central Toronto,Trail,Jewelry Store,Bus Line,Sushi Restaurant,Yoga Studio,Distribution Center,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room,4
